# KepcoDriver 

Here I will test code that will go into KepcoDriver.py

**Notes**

* I am using a 2 ohm resistor for testing purposes. 
* I need to code in a safety mechanism to not let the supply get to a dangerously high voltage/current.
* I should look into the message bounce. Now (2/4/20) it doesn't seem to be doing it so my code is breaking.
* BIG: IF the outp function goes off the supply will look like it works but then display and output nothing. Learned this the hard way. Maybe when an instance of Kepco bop is made I should "*rst" and then "OUTP ON". To make sure it's good to go. 
* 2/10/20: Get the mode thing to work, weird that you have to set a voltage first but seems to be working now as I want it. Get it to work on the current side of things and then get the flipping to work. Also getVOLT gets what the voltage has been set to but not what it truly is. How do I get what it actually is? 

In [1]:
#!/usr/bin/env python2
import socket
from socket import socket, AF_INET, SOCK_DGRAM, SOCK_STREAM

class dsocket(socket):
    def sendMessage(self,message):
        try: 
            self.send(str.encode(message))
        except RuntimeError: 
            raise RuntimeError("Error: Kepco PS not responding")

class KepcoBIT802E():
    
    
    def __init__(self, dsoc_desc=("10.66.192.42",5025)):
        """
        Initialize a Kepco Power Supply controlled via the BIT 802E Digital Interface Card; 
        open a data socket from the client side for communication
        Note that after recieving a command the BIT 802E responds first with the command itself.
        This first response is denoted message_bounce.
        ----------Parameters----------
        dsoc_desc (tuple): A tuple with the first element the IP address
            and the second element the port where data is to be received. 
        """
        
        self.ds = dsocket(AF_INET,SOCK_STREAM) # Note: SOCK_STREAM is default 
        if not dsoc_desc is None:
            try:
                self.ds.connect(dsoc_desc)
            except RuntimeError:
                raise RuntimeError("Error: Unable to connect to server at {0}".format(dsoc_desc))
        
        # I need to think about if this is the right place for this. 
        
        self.ds.sendMessage("*RST\r\n")
#         self.ds.sendMessage("OUTP ON\r\n")
    

    def setMode_VOLT(self, MaxCurr):
        """
        Sets Kepco Mode to VOLT. VOLT Mode means the supply is Voltage controlled;
        If you set MaxCurr = .1 A then you can setVolt(x) so long as voltage x doesn't create a current 
        larger than MaxCurr. Note that an x that puts the current near .1 A will 
        cause the output voltage to be less than x. 
        """
        self.ds.sendMessage("FUNC:MODE VOLT") 
        self.ds.sendMessage("VOLT 0;:OUTP ON\r\n") # For some reason you need to send it a volt command first. 
        self.ds.sendMessage("CURR {}\r\n".format(MaxCurr)) 

        
        return None     
    
    def setMode_CURR(self, MaxVolt):
        """
        Sets Kepco Mode to Curr. See comments for setMode_VOLT
        """
        self.ds.sendMessage("FUNC:MODE CURR") 
        self.ds.sendMessage("CURR 0;:OUTP ON\r\n") # For some reason you need to send it a volt command first. 
        self.ds.sendMessage("VOLT {}\r\n".format(MaxVolt)) 

        
        return None  
   
    def setVolt(self, Volt):
        """
        Returns message sent. 
        """
        self.ds.sendMessage("VOLT {};:OUTP ON\r\n".format(Volt)) 
    
        return None
   

    def setCurr(self, Curr):
        """
        Returns message sent. Note that I should think about making this verify it has worked...
        """
        self.ds.sendMessage("CURR {};:OUTP ON\r\n".format(Curr))  
    
        return None  
    
    
    def getVolt(self):
        """
        Returns string representing KEPCO voltage in Volts
        """
        self.ds.sendMessage("MEAS:VOLT?\r\n")
        response = self.ds.recv(100) 
        return response

    
    def getCurr(self):
        """
        Returns string representing KEPCO current in Amps
        """
        self.ds.sendMessage("MEAS:CURR?\r\n")                       
        response = self.ds.recv(100)
        return response
    
    
    def Message_return(self, message):
        """
        Returns string representing KEPCO current in Amps
        """
        self.ds.sendMessage("{}\r\n".format(message))                      
        response = self.ds.recv(100) 
        return response

    
    def Message_NOreturn(self, message):
        """
        Returns string representing KEPCO current in Amps
        """
        self.ds.sendMessage("{}\r\n".format(message))                      
        return None
   


# Testing Slewing of the Field: 04/15/21

To start VPN into CENPA Network: 

cd He6CRES/CENPA_VPN

sudo openvpn --config CENPADrewByron.ovpn

(mypw, cenpa)

## Constant Current Trap: 

In [11]:
from KepcoDriver import KepcoBIT802E

In [12]:
driver = KepcoBIT802E()


In [13]:
# Example of how this works: 

driver.setMode_Curr(MaxCurr = 1)
driver.setCurr(-.7)
print(driver.getCurr())

b'-6.9974E-1\r\n'


In [9]:
from SlewingFunctions import SlewFunction_Curr

In [10]:
CompTime = 0.0008
T_up =  .1     # s
T_down = .1 - CompTime  # s
duration = 1000   # s
CurrValUp = -.6   # A
CurrValDown = .1  # A
MaxCurr = .8      # A
T_stamps = SlewFunction_Curr(T_up,T_down,duration,CurrValUp,CurrValDown, MaxCurr)

KeyboardInterrupt: 

In [15]:
import numpy as np
T_stamps = np.asarray(T_stamps)
print(T_stamps.shape)
Intervals = T_stamps[1:]-T_stamps[:-1]
print(Intervals.mean(), Intervals.std())

(500,)
0.20026262728628033 0.00047633259412158477


# Testing Warming of the Trap Coil: 

Parameters: 

* 30 mins. 
* Slewing from +.5 A to -.1A. 
* Initial Temp: 52.695 K
* Final Temp: 52.772 
* Note: I don't think it really worked... Seems like the slewing timed out. 
